In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv


In [2]:
data_dir = '/kaggle/input/word2vec-nlp-tutorial/'

In [3]:
import zipfile
with zipfile.ZipFile(data_dir+'labeledTrainData.tsv.zip', 'r') as zip_ref:
    zip_ref.extractall() 
with zipfile.ZipFile(data_dir+'unlabeledTrainData.tsv.zip', 'r') as zip_ref:
    zip_ref.extractall()
with zipfile.ZipFile(data_dir+'testData.tsv.zip', 'r') as zip_ref:
    zip_ref.extractall() 

In [4]:
from IPython.display import display

In [5]:
train_df = pd.read_csv('labeledTrainData.tsv', delimiter='\t')
unlabeled_df = pd.read_csv('unlabeledTrainData.tsv', error_bad_lines=False, delimiter='\t')
test_df = pd.read_csv('testData.tsv', delimiter='\t')

display(train_df.head())
display(unlabeled_df.head())
display(test_df.head())

b'Skipping line 43043: expected 2 fields, saw 3\n'


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [6]:
import os
import time
import random
import pandas as pd
import numpy as np

import torch
from torch import nn, optim
from torch.functional import F
from torch.autograd import Variable
import torchtext
from torchtext import data
from torchtext import datasets

In [7]:
LABEL = data.LabelField()
REVIEW = data.Field(tokenize='spacy', lower=True)

In [8]:
fields_train = [('id', None),('sentiment', LABEL),('review', REVIEW)]
fields_test = [('id', None),('review', REVIEW)] 

In [9]:
twitterDataset = torchtext.data.TabularDataset(
        path="labeledTrainData.tsv",
        format="TSV",
        fields=fields_train,
        skip_header=True)

test = torchtext.data.TabularDataset(
        path="testData.tsv",
        format="TSV",
        fields=fields_test,
        skip_header=True)

In [10]:
(train, valid) = twitterDataset.split(split_ratio=[0.8,0.2])

(len(train),len(valid),len(test))

(20000, 5000, 25000)

In [11]:
vars(train.examples[7])

{'sentiment': '1',
 'review': ['\\men',
  'of',
  'honor\\',
  '"',
  '-',
  'true',
  'story',
  'about',
  'a',
  'proud',
  'and',
  'persistent',
  'black',
  'navy',
  'diver',
  '(',
  'fabulous',
  'cuba',
  'gooding',
  'jr.',
  ')',
  'is',
  'definitely',
  'a',
  'great',
  'movie',
  'that',
  'both',
  'touches',
  'and',
  'entertains',
  'and',
  'it',
  "'s",
  'part',
  'of',
  'the',
  'absolute',
  'cream',
  'of',
  'the',
  'new',
  'millennium',
  'cinema',
  '.',
  'wonderful',
  'acting',
  'is',
  'the',
  'main',
  'reason',
  'to',
  'make',
  'this',
  'movie',
  'something',
  'truly',
  'special',
  'and',
  'pretty',
  'enjoyable',
  ',',
  'splendid',
  'experience',
  '.',
  'charismatic',
  'robert',
  'de',
  'niro',
  'is',
  'marvelous',
  'as',
  'rough',
  ',',
  'fierce',
  'and',
  'pitiless',
  'chief',
  'billy',
  'sunday',
  '-',
  'role',
  'is',
  'practically',
  'written',
  'for',
  'him',
  '.',
  'this',
  'film',
  'alongside',
  'wi

In [12]:
vocab_size = 80000
REVIEW.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)
len(REVIEW.vocab)

80002

In [13]:
REVIEW.vocab.freqs.most_common(10)

[('the', 260087),
 (',', 218753),
 ('.', 187732),
 ('and', 130581),
 ('a', 129361),
 ('of', 116581),
 ('to', 108183),
 ('is', 88200),
 ('it', 74548),
 ('in', 74260)]

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test), sort = False, 
                                                                           batch_size = 128, device = device)

In [15]:

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [16]:
INPUT_DIM = len(REVIEW.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,092,105 trainable parameters


In [18]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [19]:
INPUT_DIM = len(REVIEW.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.review).squeeze(1)
        
        loss = criterion(predictions, batch.sentiment.float())
        
        acc = binary_accuracy(predictions, batch.sentiment)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.review).squeeze(1)
            
            loss = criterion(predictions, batch.sentiment.float())
            
            acc = binary_accuracy(predictions, batch.sentiment)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [22]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [23]:
N_EPOCHS = 25

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 17s
	Train Loss: 0.699 | Train Acc: 49.84%
	 Val. Loss: 0.693 |  Val. Acc: 50.27%
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.695 | Train Acc: 49.52%
	 Val. Loss: 0.694 |  Val. Acc: 50.31%
Epoch: 03 | Epoch Time: 0m 16s
	Train Loss: 0.708 | Train Acc: 49.46%
	 Val. Loss: 0.695 |  Val. Acc: 49.63%
Epoch: 04 | Epoch Time: 0m 17s
	Train Loss: 0.696 | Train Acc: 49.85%
	 Val. Loss: 0.699 |  Val. Acc: 50.94%
Epoch: 05 | Epoch Time: 0m 17s
	Train Loss: 0.695 | Train Acc: 50.81%
	 Val. Loss: 0.692 |  Val. Acc: 52.40%
Epoch: 06 | Epoch Time: 0m 16s
	Train Loss: 0.698 | Train Acc: 49.68%
	 Val. Loss: 0.693 |  Val. Acc: 50.04%
Epoch: 07 | Epoch Time: 0m 16s
	Train Loss: 0.696 | Train Acc: 49.53%
	 Val. Loss: 0.693 |  Val. Acc: 50.25%
Epoch: 08 | Epoch Time: 0m 17s
	Train Loss: 0.695 | Train Acc: 50.41%
	 Val. Loss: 0.693 |  Val. Acc: 49.80%
Epoch: 09 | Epoch Time: 0m 17s
	Train Loss: 0.695 | Train Acc: 50.18%
	 Val. Loss: 0.694 |  Val. Acc: 50.23%
Epoch: 10 | Epoch T